In [ ]:
# import plotly graph objects
import plotly.graph_objs as go

# import pandas
import pandas as pd

### Task 1: Create a scatterplot with a 1000 random data points. 

- x-axis values should come from a normal distribution using np.random.randn(1000)

- y-axis values should come from a uniform distribution over [0,1) using np.random.rand(1000)

In [ ]:
# obtain x and y values:
random_x = np.random.randn(1000) # normal distribution
random_y = np.random.rand(1000)  # uniform distribution

# define a data variable
data = [go.Scatter(
    x = random_x,
    y = random_y,
    mode = 'markers',
)]

# define the layout, and include a title and axis labels
layout = go.Layout(
    title = 'Random Data Scatterplot', # Graph title
    xaxis = dict(title = 'Normal distribution'), # x-axis label
    yaxis = dict(title = 'Uniform distribution'), # y-axis label
    hovermode ='closest' # handles multiple points landing on the same vertical
)

# Create a fig from data and layout, and plot the fig
fig = go.Figure(data=data, layout=layout)
fig.show()

-------------
### Task 2: Create a stacked bar chart with the data in the link below. 

**Note** that the questions appear in the index and should be used for the x-axis, while the responses appear as column labels. 

Stretch Task: Create a horizontal bar chart! Use this documentation [here](https://plot.ly/python/horizontal-bar-charts/) for help.

In [ ]:
# load data
df = pd.read_csv('https://raw.githubusercontent.com/Pierian-Data/Plotly-Dashboards-with-Dash/master/Data/mocksurvey.csv',index_col=0)

In [ ]:
# create traces using a list comprehension:
data = [go.Bar(
    y = df.index,     # reverse your x- and y-axis assignments
    x = df[response],
    orientation='h',  # this line makes it horizontal!
    name=response
) for response in df.columns]

# create a layout, remember to set the barmode here
layout = go.Layout(
    title='Mock Survey Results',
    barmode='stack'
)

# create a fig from data & layout, and plot the fig.
fig = go.Figure(data=data, layout=layout)
fig.show()

-------------
### Task 3: Create a histogram that plots the `length` field from dataset below. Set the range from 0 to 1, with a bin size of 0.02.

In [ ]:
# load data
df = pd.read_csv('https://raw.githubusercontent.com/Pierian-Data/Plotly-Dashboards-with-Dash/master/Data/abalone.csv')

In [ ]:
# create a data variable:
data = [go.Histogram(
    x=df['length'],
    xbins=dict(start=0,end=1,size=.02),
)]

# add a layout
layout = go.Layout(
    title="Shell lengths from the Abalone dataset"
)

# create a fig from data & layout, and plot the fig
fig = go.Figure(data=data, layout=layout)
fig.show()

## Stretch

As mentioned in the Compass, we will be using data from international football (soccer :)) matches that took place between 1872 and 2019 (148 years). You can download the dataset from [**this link**](https://drive.google.com/file/d/1cCn5botBKzh1XZOvrxpcLle-Ua7Fh9BR/view?usp=sharing) and find more information about it on [**Kaggle**](https://www.kaggle.com/martj42/international-football-results-from-1872-to-2017).

We need to make sure we understand all variables and information they store before we start working on the task. It's very important to understand the dataset to create meaningful visualizations.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import seaborn as sns
from collections import Counter

In [ ]:
df = pd.read_csv('results.csv')

In [ ]:
df.head()

## Task
Which teams scored the largest number of goals in FIFA World Cup?

In [ ]:
df_world_cup = df[df['tournament'] == 'FIFA World Cup']

In [ ]:
# top 7 teams with largest number of goals
df_plot = pd.concat([df_world_cup[['away_team','away_score']].rename(columns = {'away_team': 'team', 'away_score':'score'}),
                     df_world_cup[['home_team','home_score']].rename(columns = {'home_team': 'team', 'home_score':'score'})])\
            .groupby('team')[['score']].sum()\
            .sort_values('score',ascending=False)\
            .reset_index()\
            .head(7)


In [ ]:
fig,ax = plt.subplots(figsize=(10,10))
ax.bar(df_plot['team'], df_plot['score'])
ax.set_xlabel('Team')
ax.set_ylabel('Scored goals')
ax.set_title('World cup goals scored')
plt.show()

## Task
Show the trend in number of matches per year.

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df_plot = df['date'].dt.year.value_counts()\
                    .rename_axis('year')\
                    .to_frame('count_matches')\
                    .reset_index()\
                    .sort_values('year')

In [ ]:
data = [go.Scatter(x=df_plot['year'], y=df_plot['count_matches'])]
layout = go.Layout(title = 'Count matches per year', # Graph title
                   xaxis = dict(title = 'Year'), # x-axis label
                   yaxis = dict(title = 'Matches played')) # y-axis label
fig = go.Figure(data = data, layout = layout)
fig.show()

## Task
What is the number of matches played in each tournament throughout history?

In [ ]:
df_plot = df['tournament'].value_counts()\
                          .rename_axis('tournament')\
                          .to_frame('matches')\
                          .reset_index()\
                          .head(10)

In [ ]:
data = [go.Bar(x=df_plot['tournament'], y=df_plot['matches'])]
layout = go.Layout(title = 'Tournament matches', # Graph title
                   xaxis = dict(title = 'Tournament'), # x-axis label
                   yaxis = dict(title = 'Matches played')) # y-axis label
fig = go.Figure(data = data, layout = layout)
fig.show()

## Task
Which teams are the most successful ones? (winning percentage)

In [ ]:
def winner_is(row):
    if row['home_score'] > row['away_score']:
        return row['home_team']
    elif row['home_score'] < row['away_score']:
        return row['away_team']
    else:
        return 'draw'

In [ ]:
df['winner'] = df.apply(winner_is, axis=1)\

In [ ]:
df_count_wins = df['winner'].value_counts()\
                            .rename_axis('team')\
                            .to_frame('wins')\
                            .reset_index()\
                            .query('team != "draw"')

In [ ]:
df_matches_played = df['home_team'].value_counts().rename_axis('team').to_frame('matches').add(
                    df['away_team'].value_counts().rename_axis('team').to_frame('matches'), fill_value=0)\
                    .reset_index()

In [ ]:
df_plot = df_count_wins.merge(df_matches_played, left_on = 'team', right_on = 'team', how = 'inner')
df_plot['pct_wins'] = df_plot['wins'].divide(df_plot['matches']) * 100

In [ ]:
df_plot = df_plot.sort_values('pct_wins',ascending=False)

In [ ]:
df_plot

In [ ]:
plt.figure(figsize=(16, 6))
ax = sns.barplot(x="team", y="pct_wins", data=df_plot.head(10))
ax.set_xlabel('Team')
ax.set_ylabel('Winning percentage')
plt.show()


## Task
Which teams are the least successful ones? (winning percentage)

In [ ]:
plt.figure(figsize=(16, 6))
ax = sns.barplot(x="team", y="pct_wins", data=df_plot.tail(10))
ax.set_xlabel('Team')
ax.set_ylabel('Winning percentage')
plt.show()